In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
import requests                  
from bs4 import BeautifulSoup    
import re                        
from datetime import datetime    
from time import mktime          
import math

In [2]:
##variables
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-summary"
token = "22990f9b4amsh4df04e987b36e1bp18331cjsn862ed02f8065"
day_begin = "01-01-1990"
day_end = "01-01-2019"
interval = "1d"

In [13]:
#activate the load data function
def get_data_rollin(Stocknames, Shortnames):
    index = 0
    oop = {}
    for names in Shortnames:
        Stockname = Stocknames[index]
        index +=1
        oop[Stockname] = get_info(names)
    return oop
    
    
def get_info(Stock):
    Stock = load_csv_data(Stock, interval, day_begin, day_end)
    if len(Stock) < 1000:
        Stock = load_csv_data(Stock, interval, day_begin, day_end)
    return Stock

In [4]:
def _get_crumbs_and_cookies(stock):
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

In [5]:
def convert_to_unix(date):
    
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


In [6]:
def load_csv_data(stock, interval, day_begin, day_end):
  
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
        if len(website.text) < 1000:
            print("something with the data of",stock,"went wrong")
        else: print("mhm we've just received a shit ton of data from",stock)
        
        return website.text.split('\n')[:-1]
    
    

In [7]:
def delete_wrong_datasets(List_of_raws, compname):
    index = 0 
    companynames = list()
    for x in list(List_of_raws.values()):
        if len(List_of_raws[compname[index]]) < 500:
            #print(len(List_of_raws[compname[index]]))
            del List_of_raws[compname[index]]
            index += 1
        else:
            companynames.append(compname[index])
            index += 1
    return List_of_raws
            
            

In [8]:
def new_comp_names(List_of_raws, compnames):
    index = 0 
    updated_compnames = list()
    for x in List_of_raws:
        if x in compnames:
            updated_compnames.append(x)
    return updated_compnames
            

In [9]:
def getting_data_seperated(List_of_raws, compname):
    list2 = list()
    compnameindex = -1
    for x in List_of_raws.values():
        compnameindex += 1  
        temp = list()
        for z in x:
            temp.append(z.split(","))
            new_cols = ["Date"] 
        for name in temp[0]:
            if name != "Date":
                name = compname[compnameindex] + name
                new_cols.append(name)
        temp[0] = new_cols
        list2.append(temp)
        
    list2 = creating_dfs(Startprocess[0], list2)
    
    return list2
#df2 = pd.DataFrame(list2)
#df2.transpose()

In [10]:
def creating_dfs(Compnames, List):
    df_all = pd.DataFrame(List[0])
    df_all = df_all.rename(columns=df_all.iloc[0])
    df_all = df_all.drop(0,axis=0)
    index = 1 
    while index < len(List):
        df = pd.DataFrame(List[index]) 
        df = df.rename(columns=df.iloc[0])
        df = df.drop(0,axis=0)
        df_all = pd.merge(df_all,df, on="Date") 
        index += 1

    return df_all

In [11]:
#START THE PROCESS
def start():
    Stockslist = list()
    Stocknames = list()
    End_of_Party = 0
    inputquest = input("Do you want to insert a list or every company on its own \n list/single")
    if inputquest == "list":
        stocklistinsert = input("Paste your Stocklist :)")
        Stocklistinsert = stocklistinsert.split(" ")
        Stocklistraw = Stocklistinsert[1::2]
        Stocklist = list()
        for x in Stocklistraw:
            Stocklist.append(x.upper())
        Stocknamesraw = Stocklistinsert[::2]
        Stocknames = list()
        for x in Stocknamesraw:
            Stocknames.append(x.upper())
            
        return Stocknames, Stocklist
    
    while End_of_Party == 0:
        input_stock = (input("Insert the Name of the Company to add stock, otherwise enter \"done\": "))
        input_stock = input_stock.upper()
        if input_stock == "DONE":
            End_of_Party += 1
        else:
            Stocknames.append(input_stock)
            input_stock = (input("Insert the SHORT NAME (Stockname) to add stock, otherwise enter \"done\": "))
            input_stock = input_stock.upper()
            if input_stock == "DONE":
                End_of_Party += 1
            else:
                Stockslist.append(input_stock)
            continue
        print(Stocknames)
        print(Stockslist)
        return Stocknames, Stockslist

In [17]:
Startprocess = start()
List_of_raws = get_data_rollin(Startprocess[0],Startprocess[1])
List_of_raws = delete_wrong_datasets(List_of_raws, Startprocess[0])
New_compnamelist = new_comp_names(List_of_raws, Startprocess[0])
df_all = getting_data_seperated(List_of_raws, New_compnamelist)
df_all

#Software        //SAP = SAP // APPLE = AAPL // AMAZON = AMZN // MICROSOFT = MSFT \n",
#Automobil      //BMW = BMW.DE // Volkswagen = VOW3.DE // Peugeot = PEU.F // Mitsubishi = MSBHY\n",
#Banks         //GOLDMAN = GS // JP MORGAN =JPM // DEUTSCHE BANK = DB // HSBC = HSBC\n",
#sonst        //STARBUCKS = SBUX // WALLMART = WMT // ADIDAS = ADS.DE // NIKE = NKE",
#INDEX       //DOW JONES = ^DJI // DAX30 = ^GDAXI // NIKKEI = ^N225 // 
#SAP', 'APPLE', 'AMAZON', 'MICROSOFT', 'BMW', 'VOLKSWAGEN', 'PEUGOT', 'MITSUBISHI', 'GOLDMAN', 'JPMORGAN', 'DEUTSCHE', 'DB', 'HSBC']
#['SAP', 'AAPL', 'AMZN', 'MSFT', 'BMW.DE', 'VOW3.DE', 'PEU.F', 'MSBHY', 'GS', 'JPM', 'BANK', 'HSBC']

#sap sap apple aapl amazon amzn microsoft msft bmw bmw.de volkswagen vow3.de peugot peu.f mitsubishi msbhy goldman gs JPMorgan jpm DEUTSCHE.BANK DB HSBC HSBC

Do you want to insert a list or every company on its own 
 list/single single
Insert the Name of the Company to add stock, otherwise enter "done":  aapl
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  aapl
Insert the Name of the Company to add stock, otherwise enter "done":  done


['AAPL']
['AAPL']
mhm we've just received a shit ton of data from AAPL


,Date,AAPLOpen,AAPLHigh,AAPLLow,AAPLClose,AAPLAdj Close,AAPLVolume
1,1990-01-02,1.258929,1.339286,1.250000,1.330357,1.088798,45799600
2,1990-01-03,1.357143,1.357143,1.339286,1.339286,1.096105,51998800
3,1990-01-04,1.366071,1.383929,1.330357,1.343750,1.099759,55378400
4,1990-01-05,1.348214,1.366071,1.321429,1.348214,1.103412,30828000
5,1990-01-08,1.339286,1.357143,1.321429,1.357143,1.110720,25393200
6,1990-01-09,1.357143,1.357143,1.321429,1.343750,1.099759,21534800
7,1990-01-10,1.343750,1.343750,1.276786,1.285714,1.052261,49929600
8,1990-01-11,1.294643,1.294643,1.232143,1.232143,1.008417,52763200
9,1990-01-12,1.223214,1.241071,1.205357,1.232143,1.008417,42974400
10,1990-01-15,1.232143,1.276786,1.223214,1.223214,1.001109,40434800
